In [34]:
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeRegressor

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor,\
                             StackingRegressor, VotingRegressor

from sklearn.svm import LinearSVR, SVR
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from scipy.stats import normaltest
import sklearn.metrics


import statsmodels.api as sm 
#import pylab as py 

from scipy.stats import boxcox
from scipy.special import inv_boxcox
#import autosklearn.regression
# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 7, 4

import manufacturing as mn
import visual, analysis

%matplotlib
ROOT_DIR = './'
print("Ready!")

Using matplotlib backend: Qt5Agg
Ready!


In [2]:
def round_to_hour(dt):
    if dt.minute > 30:#date.replace(minute = 0, hour = 0)
        if dt.hour == 23:
            return dt.replace(minute = 0, hour = 0, second = 0)
        else:
            return dt.replace(minute = 0, hour = dt.hour + 1, second = 0) 
    else:
        return dt.replace(minute = 0, second = 0)
    
def round_dt_to_10min(dt):
    try:
        tmp_min = int(str(int(dt.minute/10)) + '0') if dt.minute/10 > 1 else int(dt.minute)
    except ValueError as e:
        print(e, dt)
        return 'Null'
    if (dt.minute - tmp_min) < 5:
        return dt.replace(minute = tmp_min, second = 0)#если разница меньше 5 минут
    else: 
        try:
            if dt.hour == 23 and tmp_min == 50:
                return dt.replace(day = dt.day + 1 ,minute = 0, second = 0, hour = 0) #для 23:55+ переход на некст день
        except ValueError:
            try:
                return dt.replace(day = 1 ,minute = 0, second = 0, hour = 0, month = dt.month + 1) #для 23:55+ переход на некст месяц
            except ValueError:
                return dt.replace(day = 1 ,minute = 0, second = 0, hour = 0, month = 1, year = dt.year + 1)
        if tmp_min == 50:
            return dt.replace(minute = 0, second = 0, hour = dt.hour + 1)#для НН:55+ добавляем час
        return dt.replace(minute = tmp_min + 10, second = 0)
    
def round_to_half_hour(dt):
    #q = int(dt.minute/15)
    #print(q, dt)
    td = timedelta(hours = 1)
    if dt.minute <= 15:
        return dt.replace(minute = 0)
    if dt.minute > 15 & dt.minute < 45:
        return dt.replace(minute = 30)
    if dt.minute >= 45:
        return dt.replace(minute = 0) + td

def is_norm(s):
    #Тест д'Агостино на K ^ 2 
    #if p < a:  # null hypothesis: x comes from a normal distribution
    a = 0.05
    sts, p = normaltest(s)
    if p > a:
        #print('+Sample looks Gaussian (fail to reject H0)')
        return True, p
    else:
        #print('-Sample does not look Gaussian (reject H0)',p)
        return False, p
    
#http://statistica.ru/local-portals/quality-control/indeksy-prigodnosti-protsessa/
def calc_stats(s):
    #Up и Lp – 99.865 и 0.135 процентили соответственно
    Up = s.quantile(q = 0.99865)
    Lp = s.quantile(q = 0.00135)
    M = s.median()
    return Up, Lp, M

def calc_pp(s, NGD, VGD):
    Up, Lp, _  = calc_stats(s)
    #Потенциальная пригодность (Cp)
    Pp = (VGD-NGD)/(Up-Lp) 
    return Pp

def calc_ppu(s, NGD, VGD):
    Up, Lp, M  = calc_stats(s)
    Ppu = (VGD-M)/(Up - M) 
    #Подтвержденное качество (Cpk)

    return Ppu

def calc_ppl(s,NGD,VGD):
    Up, Lp, M = calc_stats(s)
    Ppl = (M - NGD)/(M - Lp) 
    return Ppl


def calc_ppk(s, NGD, VGD):
    return min((calc_ppl(s, NGD, VGD), calc_ppu(s, NGD, VGD)))


In [3]:
def prepare_mes_df(mes_df):
    mes_df = mes_df.replace('Null', np.nan)
    mes_df['34апп.сумма.3нитка'] = mes_df['DK14_SKD_Deg_003.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_003.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_003.FRC3009'].astype(float)
    mes_df['34апп.сумма.4нитка'] = mes_df['DK14_SKD_Deg_004.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_004.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_004.FR3012'].astype(float)
    mes_df['34апп.сумма.5нитка'] = mes_df['DK14_SKD_Deg_005.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_005.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_005.FR3015'].astype(float)

    #подача сухого полимера
    mes_df['DK14_SKD_Deg_003.F1'] /=  10
    mes_df['DK14_SKD_Deg_004.F1'] /= 10
    mes_df['DK14_SKD_Deg_005.F1'] /= 10

    mes_df['Суммарная подача пара на 3 нитку, т/час'] = mes_df['DK14_SKD_Deg_003.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_003.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_003.FRC3009'].astype(float) + mes_df['DK14_SKD_Deg_003.F3_3'].astype(float)
    mes_df['Суммарная подача пара на 4 нитку, т/час'] = mes_df['DK14_SKD_Deg_004.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_004.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_004.FR3012'].astype(float) + mes_df['DK14_SKD_Deg_004.F3_3'].astype(float)
    mes_df['Суммарная подача пара на 5 нитку, т/час'] = mes_df['DK14_SKD_Deg_005.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_005.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_005.FR3015'].astype(float) + mes_df['DK14_SKD_Deg_005.F3_3'].astype(float)

    formula = 'пар/полимер, ед.изм = ГКал\т'
    mes_df['34апп.удел.сумма.3нитка'] = ((mes_df['DK14_SKD_Deg_003.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_003.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_003.FRC3009'].astype(float)))*0.67/mes_df['DK14_SKD_Deg_003.F1'].astype(float)
    mes_df['34апп.удел.сумма.4нитка'] = ((mes_df['DK14_SKD_Deg_004.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_004.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_004.FR3012'].astype(float)))*0.67/mes_df['DK14_SKD_Deg_004.F1'].astype(float)
    mes_df['34апп.удел.сумма.5нитка'] = ((mes_df['DK14_SKD_Deg_005.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_005.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_005.FR3015'].astype(float)))*0.67/mes_df['DK14_SKD_Deg_005.F1'].astype(float)
    mes_df['удел.сумм.3нитка'] = (mes_df['Суммарная подача пара на 3 нитку, т/час'].astype(float))*0.67/mes_df['DK14_SKD_Deg_003.F1'].astype(float)
    mes_df['удел.сумм.4нитка'] = (mes_df['Суммарная подача пара на 4 нитку, т/час'].astype(float))*0.67/mes_df['DK14_SKD_Deg_004.F1'].astype(float)
    mes_df['удел.сумм.5нитка'] = (mes_df['Суммарная подача пара на 5 нитку, т/час'].astype(float))*0.67/mes_df['DK14_SKD_Deg_005.F1'].astype(float)
    mes_df['120апп.удел.3нитка'] = (mes_df['DK14_SKD_Deg_003.F3_3'].astype(float))*0.67/mes_df['DK14_SKD_Deg_003.F1'].astype(float)
    mes_df['120апп.удел.4нитка'] = (mes_df['DK14_SKD_Deg_004.F3_3'].astype(float))*0.67/mes_df['DK14_SKD_Deg_004.F1'].astype(float)
    mes_df['120апп.удел.5нитка'] = (mes_df['DK14_SKD_Deg_005.F3_3'].astype(float))*0.67/mes_df['DK14_SKD_Deg_005.F1'].astype(float)

    
    #Сдвиг во времени от дегазаторов до места взячи анализа
    mes_df['Дата и время'] = mes_df['Дата и время'].astype('datetime64[ns]') - timedelta(minutes = 90)
    
    
    return mes_df

def prepare_lims_df(lims_df):
    try:
        lims_df = lims_df.drop(columns = ['Unnamed: 3','Unnamed: 4','Массовая доля антиоксиданта Агидол-2, Проценты относител.','Unnamed: 6','Unnamed: 8','Unnamed: 9','Unnamed: 11','Unnamed: 12', 'Unnamed: 13','Примечание','Место отбора', "Номер партии"]).rename(columns = {'Дата и время отбора':'Дата и время'}).iloc[:,:3]
    except KeyError:
        lims_df = lims_df.drop(columns = ['Unnamed: 3','Unnamed: 4','Unnamed: 6','Unnamed: 8','Unnamed: 9','Примечание','Место отбора', "Номер партии"]).rename(columns = {'Дата и время отбора':'Дата и время'}).iloc[:,:3]
    lims_df['Массовая доля тримеров, %'] = (lims_df['Массовая доля тримеров, %']/1000).astype(float)
    lims_df['Массовая доля углеводородов, %'] = (lims_df['Массовая доля углеводородов, %']/100).astype(float)
    lims_df['Дата и время'] = lims_df['Дата и время'].astype('datetime64[ns]').apply(round_to_half_hour)
    
    #tdelta = timedelta(minutes = 30)
    #lims_df['Дата и время'] = lims_df['Дата и время'] + tdelta
    return lims_df

def make_final_df(mes_df, lims_df):
    fin_df = mes_df.merge(lims_df, how = 'left', on = 'Дата и время').sort_values(by='Дата и время')
    #


    return fin_df


In [4]:
lims_data = pd.read_excel(os.path.join(ROOT_DIR, 'Сводный журнал результатов анализов за период ЦЗЛ ПП ПБК.xlsx'),sheet_name = 'Лист1',thousands = ',')
mes_data = pd.read_excel(os.path.join(ROOT_DIR, 'скд.xlsx'),sheet_name = 'скд').drop(columns = [ 'wwRetrievalMode','Дата и время.1'])

addition_data = pd.read_excel("C:\\Users\\bobca\\WorkData\\SKD_DEG\\skd deg new data\\Лист Microsoft Excel.xlsx",sheet_name = 'Лист1').replace('Null', np.nan).drop(columns = ['wwRetrievalMode'])
addition_data['Дата и время'] = addition_data['Дата и время'].astype('datetime64[ns]') - timedelta(minutes = 90)

mes_df1 = prepare_mes_df(mes_data)
lims_df1 = prepare_lims_df(lims_data)
mes_df1 = mes_df1.merge(addition_data, how = 'left', on = 'Дата и время')
final_df1 = make_final_df(mes_df1, lims_df1)


lims_new_data = pd.read_excel("C:\\Users\\bobca\\WorkData\\SKD_DEG\\skd deg new data\\Сводный журнал результатов c 18 до 19,5.xlsx",sheet_name = 'Лист2',thousands = ',')
mes_new_data = pd.read_excel("C:\\Users\\bobca\\WorkData\\SKD_DEG\\skd deg new data\\skd18-195_new.xlsx",sheet_name = 'skd18-195_new')
mes_df2 = prepare_mes_df(mes_new_data)
lims_df2 = prepare_lims_df(lims_new_data)
final_df2 = make_final_df(mes_df2, lims_df2)

fin_df = final_df1.append(final_df2)

muni_df =  pd.read_excel(os.path.join(ROOT_DIR, 'Рабочий журнал определения вязкости по Муни (полимеризат).xlsx'),sheet_name = 'sheet1',thousands = ',').drop(columns = ['Номер партии','№ тех. линии','Unnamed: 4','Unnamed: 7','Unnamed: 8','Unnamed: 9','ФИО \nисполнителя','Unnamed: 12'])
muni_df['Муни'] = muni_df['Муни']/100
def prs(str_d):
    return datetime.strptime(str_d, '%d:%m:%Y %H:%M')
muni_df['Дата и время'] = muni_df['Дата отбора'] + ' ' + muni_df['Время отбора']
muni_df['Дата и время'] = muni_df['Дата и время'].apply(prs).astype('datetime64[ns]').apply(round_to_half_hour)
muni_df = muni_df.drop(columns = ['Дата отбора','Время отбора'])
muni_df['Дата и время'] = muni_df['Дата и время'].astype('datetime64[ns]') + timedelta(minutes = 30)

fin_df = fin_df.merge(muni_df, how = 'left', on ='Дата и время')

try:
    fin_df.to_excel('fin_df.xlsx')
except pd.FileCreateError as fce:
    print(fce)
fin_df

,120апп.удел.3нитка,120апп.удел.4нитка,120апп.удел.5нитка,34апп.сумма.3нитка,34апп.сумма.4нитка,34апп.сумма.5нитка,34апп.удел.сумма.3нитка,34апп.удел.сумма.4нитка,34апп.удел.сумма.5нитка,DK14_SKD_Deg_003.F1,...,"Массовая доля углеводородов, %","Суммарная подача пара на 3 нитку, т/час","Суммарная подача пара на 4 нитку, т/час","Суммарная подача пара на 5 нитку, т/час",удел.сумм.3нитка,удел.сумм.4нитка,удел.сумм.5нитка,Точка отбора,Муни,Площадь релаксации
0,0.0,1.608311,1.560061,-0.048965,25.297186,26.960779,-3.957034e+15,2.806579,3.000372,8.290680e-18,...,NaN,-0.048965,39.793751,40.979198,-3.957034e+15,4.414891,4.560434,NaN,NaN,NaN
1,0.0,1.609767,1.565473,-0.052734,25.319632,27.031043,-4.261662e+15,2.814135,3.024005,8.290680e-18,...,NaN,-0.052734,39.803199,41.024525,-4.261662e+15,4.423903,4.589477,NaN,NaN,NaN
2,0.0,1.612604,1.559316,-0.051353,25.333811,27.007740,-4.150033e+15,2.820297,3.007926,8.290680e-18,...,NaN,-0.051353,39.819309,41.008613,-4.150033e+15,4.432902,4.567242,NaN,NaN,NaN
3,0.0,1.611580,1.560888,-0.048707,25.338701,27.024299,-3.936194e+15,2.813271,3.018050,8.290680e-18,...,NaN,-0.048707,39.853957,41.000843,-3.936194e+15,4.424852,4.578938,NaN,NaN,NaN
4,0.0,1.612793,1.556335,-0.050377,25.304534,27.010660,-4.071182e+15,2.815966,3.010917,8.290680e-18,...,NaN,-0.050377,39.797246,40.972400,-4.071182e+15,4.428759,4.567251,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65127,0.0,1.612468,1.553787,-0.051619,25.418296,26.900272,-4.171481e+15,2.825700,2.989259,8.290680e-18,...,NaN,-0.051619,39.923092,40.882761,-4.171481e+15,4.438168,4.543046,NaN,NaN,NaN
65128,0.0,1.610821,1.557292,-0.051103,25.450527,26.982763,-4.129798e+15,2.827195,3.004411,8.290680e-18,...,NaN,-0.051103,39.951203,40.968883,-4.129798e+15,4.438015,4.561703,NaN,NaN,NaN
65129,0.0,1.615200,1.558635,-0.051501,25.304810,26.960389,-4.161999e+15,2.818104,2.996399,8.290680e-18,...,NaN,-0.051501,39.808294,40.984361,-4.161999e+15,4.433304,4.555035,NaN,NaN,NaN
65130,0.0,1.615843,1.548354,-0.049309,25.234069,26.985618,-3.984805e+15,2.811615,2.990540,8.290680e-18,...,NaN,-0.049309,39.736163,40.957440,-3.984805e+15,4.427458,4.538894,NaN,NaN,NaN


count    64148.000000
mean        10.768528
std          0.868805
min          7.083566
25%         10.557480
50%         11.006597
75%         11.291914
max         12.758804
Name: DK14_SKD_Prod.QR1501, dtype: float64

In [66]:
#FOR ECONS
#print(fin_df.columns.to_list())

ogrs = {'T2':[110,115], 'T1':[95, 105], 'P1':[0.1, 0.5], 'P2':[0.3, 0.7],
        'F1':[4,7], 'Суммарная подача пара на 3 нитку, т/час':[38,44], 'Суммарная подача пара на 4 нитку, т/час':[38,44],
        'Массовая доля углеводородов, %':[0.01, 0.3], 'Суммарная подача пара на 5 нитку, т/час':[38,44],
        'сумм':[4,8],'VR010104EKSTEAM_NORTH.LHA':[7.4, 8.3], 'VR010104EKSTEAM_SOUTH.LHA':[5.5, 6.3]}


nitka_3_df = fin_df[[
                    'DK14_SKD_Deg_003.T1','DK14_SKD_Deg_003.T2',
                    'DK14_SKD_Deg_003.P1','DK14_SKD_Deg_003.P2',
                    'DK14_SKD_Deg_003.F1','Дата и время',
                    'Суммарная подача пара на 3 нитку, т/час',
                    'удел.сумм.3нитка','Массовая доля углеводородов, %',
                    'VR010104EKSTEAM_NORTH.LHA', 'VR010104EKSTEAM_SOUTH.LHA',
                    'DK14_SKD_Prod.QR1502', 'DK14_SKD_Prod.QR1501', 'DK14_SKD_Prod.Mn','DK14_SKD_Prod.SO',
                   ]][\
                       (fin_df['DK14_SKD_Deg_003.F1'] > 1)
                     & (fin_df['Суммарная подача пара на 3 нитку, т/час'] > 5)
                     & (fin_df['34апп.удел.сумма.3нитка'] < 8)
                     & (fin_df['удел.сумм.3нитка'] < 6.5)
                     & (fin_df['DK14_SKD_Deg_003.T1'] > 85)
                     & (fin_df['DK14_SKD_Deg_003.T1'] < 115)
                     & (fin_df['DK14_SKD_Deg_003.T2'] > 100)
                     & (fin_df['DK14_SKD_Deg_003.T2'] < 130)
                     & (fin_df['DK14_SKD_Deg_003.P1'] > 0.15)
                     & (fin_df['DK14_SKD_Deg_003.P1'] < 0.35)                      
                     & (fin_df['DK14_Shop.REZIM_3N'] == 0)
                     & (fin_df['DK14_SKD_Prod.QR1501'] > 8)
                     & (fin_df['DK14_SKD_Prod.QR1502'] < 10)                      
                     & (fin_df['Дата и время'] > datetime(2020,1,1,0,0))\
                    ].reset_index().drop(columns = 'index')

#pandas_profiling.ProfileReport(df = nitka_3_df.drop(columns = 'Дата и время')).to_file('3.html')


nitka_4_df = fin_df[[
                    'DK14_SKD_Deg_004.T1','DK14_SKD_Deg_004.T2',
                    'DK14_SKD_Deg_004.P1','DK14_SKD_Deg_004.P2',
                    'DK14_SKD_Deg_004.F1','Дата и время',
                    'Суммарная подача пара на 4 нитку, т/час',
                    'удел.сумм.4нитка','Массовая доля углеводородов, %',
                    'VR010104EKSTEAM_NORTH.LHA', 'VR010104EKSTEAM_SOUTH.LHA',
                    'DK14_SKD_Prod.QR1502', 'DK14_SKD_Prod.QR1501', 'DK14_SKD_Prod.Mn','DK14_SKD_Prod.SO'
                   ]][\
                       (fin_df['DK14_SKD_Deg_004.F1'] > 1)
                     & (fin_df['Суммарная подача пара на 4 нитку, т/час'] > 5)
                     & (fin_df['34апп.удел.сумма.4нитка'] < 8)
                     & (fin_df['удел.сумм.4нитка'] < 6.5)
                     & (fin_df['DK14_SKD_Deg_004.T2'] > 90)
                     & (fin_df['DK14_SKD_Deg_004.T2'] < 130)
                     & (fin_df['DK14_SKD_Deg_004.T1'] > 85)
                     & (fin_df['DK14_SKD_Deg_004.T1'] < 115)
                     & (fin_df['DK14_SKD_Deg_004.P1'] > 0.15)
                     & (fin_df['DK14_SKD_Deg_004.P1'] < 0.35)
                     & (fin_df['DK14_SKD_Prod.QR1501'] > 8)
                     & (fin_df['DK14_SKD_Prod.QR1502'] < 10)                      
                     & (fin_df['Дата и время'] > datetime(2020,1,1,0,0))\
                    ].reset_index().drop(columns = 'index')

#pandas_profiling.ProfileReport(df = nitka_4_df.drop(columns = 'Дата и время')).to_file('4.html')

nitka_5_df = fin_df[[
                    'DK14_SKD_Deg_005.T1','DK14_SKD_Deg_005.T2',
                    'DK14_SKD_Deg_005.P1','DK14_SKD_Deg_005.P2',
                    'DK14_SKD_Deg_005.F1','Дата и время',
                    'Суммарная подача пара на 5 нитку, т/час',
                    'удел.сумм.5нитка','Массовая доля углеводородов, %',
                    'VR010104EKSTEAM_NORTH.LHA', 'VR010104EKSTEAM_SOUTH.LHA',
                    'DK14_SKD_Prod.QR1502', 'DK14_SKD_Prod.QR1501', 'DK14_SKD_Prod.Mn','DK14_SKD_Prod.SO'
                   ]][  (fin_df['DK14_SKD_Deg_005.F1'] > 1)
                     & (fin_df['Суммарная подача пара на 5 нитку, т/час'] > 5)
                     & (fin_df['34апп.удел.сумма.5нитка'] < 8)
                     & (fin_df['удел.сумм.5нитка'] < 6.5)
                     & (fin_df['DK14_SKD_Deg_005.T2'] < 130)
                     & (fin_df['DK14_SKD_Deg_005.T2'] > 100)
                     & (fin_df['DK14_SKD_Deg_005.T1'] > 85)
                     & (fin_df['DK14_SKD_Deg_005.T1'] < 115)
                     & (fin_df['DK14_SKD_Deg_005.P1'] > 0.15)
                     & (fin_df['DK14_SKD_Deg_005.P1'] < 0.45)
                     & (fin_df['DK14_SKD_Prod.QR1501'] > 8)
                     & (fin_df['DK14_SKD_Prod.QR1502'] < 10)
                     & (fin_df['Дата и время'] > datetime(2020,1,1,0,0))\
                    ].reset_index().drop(columns = 'index') 
#pandas_profiling.ProfileReport(df = nitka_5_df.drop(columns = 'Дата и время')).to_file('5.html')

print('nitka_3:',nitka_3_df.shape)
print('nitka_4:',nitka_4_df.shape)
print('nitka_5:',nitka_5_df.shape)
nitkas = (nitka_3_df, nitka_4_df, nitka_5_df)

nitka_3: (10081, 15)
nitka_4: (15624, 15)
nitka_5: (17078, 15)


In [78]:
#fin_df['DK14_SKD_Prod.QR1501'][fin_df['DK14_SKD_Prod.QR1501'] < 8].describe()
#fin_df[['DK14_SKD_Prod.QR1501','Дата и время']].sort_values(by = 'Дата и время')

In [79]:
nitkas[0].columns.to_list()
r = {'DK14_SKD_Deg_005.T1':'34 дег., температура',
 'DK14_SKD_Deg_005.T2':'120 дег., температура',
 'DK14_SKD_Deg_005.P1':'34 дег. давление',
 'DK14_SKD_Deg_005.P2':'120 дег. давление',
 'DK14_SKD_Deg_005.F1':'34 дег. подача полим, т\ч',
 'VR010104EKSTEAM_NORTH.LHA':'пар СЕВЕР, давление',
 'VR010104EKSTEAM_SOUTH.LHA':'пар ЮГ, давление',
 'DK14_SKD_Prod.QR1502':'pH цирк.воды',
 'DK14_SKD_Prod.QR1501':'pH H-651-3',
 'DK14_SKD_Prod.Mn':'Муни',
 'DK14_SKD_Prod.SO':'Сухой остаток'}
top=0.98,
bottom=0.255,
left=0.18,
right=0.9,
hspace=0.2,
wspace=0.2
sns.heatmap(nitkas[2].rename(columns = r).corr(), annot = True)

<AxesSubplot:>

In [22]:
i = 1
for dftmp in nitkas:
    dftmp = dftmp.drop(columns = ['Дата и время','DK14_SKD_Prod.QR1502', 'DK14_SKD_Prod.QR1501', 'DK14_SKD_Prod.Mn','DK14_SKD_Prod.SO'])
    for col in dftmp.columns.to_list():
        try:
            print(col,i)
            i+=1
            visual.ppk_plot(dftmp[col], lower_control_limit = ogrs[col.split('.')[1]][0],\
                                  upper_control_limit =ogrs[col.split('.')[1]][1])
        except:
            visual.ppk_plot(dftmp[col], lower_control_limit = ogrs[col][0],\
                                  upper_control_limit = ogrs[col][1])        

DK14_SKD_Deg_003.T1 1
DK14_SKD_Deg_003.T2 2
DK14_SKD_Deg_003.P1 3
DK14_SKD_Deg_003.P2 4
DK14_SKD_Deg_003.F1 5
Суммарная подача пара на 3 нитку, т/час 6
удел.сумм.3нитка 7
Массовая доля углеводородов, % 8
VR010104EKSTEAM_NORTH.LHA 9
VR010104EKSTEAM_SOUTH.LHA 10
DK14_SKD_Deg_004.T1 11
DK14_SKD_Deg_004.T2 12
DK14_SKD_Deg_004.P1 13
DK14_SKD_Deg_004.P2 14
DK14_SKD_Deg_004.F1 15
Суммарная подача пара на 4 нитку, т/час 16
удел.сумм.4нитка 17
Массовая доля углеводородов, % 18
VR010104EKSTEAM_NORTH.LHA 19
VR010104EKSTEAM_SOUTH.LHA 20
DK14_SKD_Deg_005.T1 21
DK14_SKD_Deg_005.T2 22
DK14_SKD_Deg_005.P1 23
DK14_SKD_Deg_005.P2 24
DK14_SKD_Deg_005.F1 25
Суммарная подача пара на 5 нитку, т/час 26
удел.сумм.5нитка 27
Массовая доля углеводородов, % 28
VR010104EKSTEAM_NORTH.LHA 29
VR010104EKSTEAM_SOUTH.LHA 30


In [ ]:
scal = MinMaxScaler()
#s_scal = scal.fit_transform(nitkas[1].drop(columns = ['Дата и время']).to_numpy())
z = nitkas[1]['DK14_SKD_Deg_004.P1']
print(is_norm(z))
p_log = np.log(z)
#mn.ppk_plot(p_log, upper_control_limit=0.1, lower_control_limit=0.4)
mn.ppk_plot(z, upper_control_limit=0.1, lower_control_limit=0.4)
#sm.qqplot(z,loc = z.mean(), scale = z.std(), line = '45') 
#sm.qqplot(p_log, loc=p_log.mean(), scale=p_log.std(), line = '45')


y, fitted_lambda = boxcox(z, lmbda=None)
#c = inv_boxcox(y, fitted_lambda)


mn.ppk_plot(y, upper_control_limit=0.1, lower_control_limit=0.4)
sm.qqplot(y, loc= y.mean(), scale=y.std(), line = '45')


In [39]:
#Для drop'a нитки
nit3 = ['DK14_SKD_Deg_003.F1',
 'DK14_SKD_Deg_003.F1_M',
 'DK14_SKD_Deg_003.F2_1',
 'DK14_SKD_Deg_003.F2_2',
 'DK14_SKD_Deg_003.F3_1',
 'DK14_SKD_Deg_003.F3_1.IsOverheat',
 'DK14_SKD_Deg_003.F3_1.T_sat',
 'DK14_SKD_Deg_003.F3_2',
 'DK14_SKD_Deg_003.F3_2.IsOverheat',
 'DK14_SKD_Deg_003.F3_2.T_sat',
 'DK14_SKD_Deg_003.F3_3',
 'DK14_SKD_Deg_003.F4',
 'DK14_SKD_Deg_003.F5',
 'DK14_SKD_Deg_003.F6',
 'DK14_SKD_Deg_003.F7_1',
 'DK14_SKD_Deg_003.F7_2',
 'DK14_SKD_Deg_003.FRC3009',
 'DK14_SKD_Deg_003.FRC3009.IsOverheat',
 'DK14_SKD_Deg_003.FRC3009.T_sat',
 'DK14_SKD_Deg_003.I1',
 'DK14_SKD_Deg_003.I2',
 'DK14_SKD_Deg_003.L1',
 'DK14_SKD_Deg_003.L1_1',
 'DK14_SKD_Deg_003.L1_2',
 'DK14_SKD_Deg_003.L2',
 'DK14_SKD_Deg_003.L2_1',
 'DK14_SKD_Deg_003.L2_2',
 'DK14_SKD_Deg_003.P1',
 'DK14_SKD_Deg_003.P2',
 'DK14_SKD_Deg_003.P2_1',
 'DK14_SKD_Deg_003.Q2',
 'DK14_SKD_Deg_003.Q2_2',
 'DK14_SKD_Deg_003.Q3_1_2',
 'DK14_SKD_Deg_003.Q3_2',
 'DK14_SKD_Deg_003.Q3_3',
 'DK14_SKD_Deg_003.Q4',
 'DK14_SKD_Deg_003.Q5',
 'DK14_SKD_Deg_003.Q6',
 'DK14_SKD_Deg_003.State1',
 'DK14_SKD_Deg_003.State2',
 'DK14_SKD_Deg_003.T1',
 'DK14_SKD_Deg_003.T2',
 'DK14_SKD_Deg_003.T3',
 'DK14_SKD_Deg_003.T4',
 'DK14_SKD_Deg_003.TT100',
 'DK14_SKD_Deg_003.TT101',
 '34апп.сумма.3нитка',
 'Суммарная подача пара на 3 нитку, т/час',
 '34апп.удел.сумма.3нитка',
 'удел.сумм.3нитка',
 '120апп.удел.3нитка',
 'DK14_Shop.REZIM_3N']


nit4 = [ 'DK14_SKD_Deg_004.F1',
 'DK14_SKD_Deg_004.F1_M',
 'DK14_SKD_Deg_004.F2_1',
 'DK14_SKD_Deg_004.F2_2',
 'DK14_SKD_Deg_004.F3_1',
 'DK14_SKD_Deg_004.F3_1.IsOverheat',
 'DK14_SKD_Deg_004.F3_1.T_sat',
 'DK14_SKD_Deg_004.F3_2',
 'DK14_SKD_Deg_004.F3_2.IsOverheat',
 'DK14_SKD_Deg_004.F3_2.T_sat',
 'DK14_SKD_Deg_004.F3_3',
 'DK14_SKD_Deg_004.F3_3.IsOverheat',
 'DK14_SKD_Deg_004.F3_3.T_sat',
 'DK14_SKD_Deg_004.F4',
 'DK14_SKD_Deg_004.F5',
 'DK14_SKD_Deg_004.F6',
 'DK14_SKD_Deg_004.F7_1',
 'DK14_SKD_Deg_004.F7_2',
 'DK14_SKD_Deg_004.FR3012',
 'DK14_SKD_Deg_004.FR3012.IsOverheat',
 'DK14_SKD_Deg_004.FR3012.T_sat',
 'DK14_SKD_Deg_004.FRC3012',
 'DK14_SKD_Deg_004.I1',
 'DK14_SKD_Deg_004.I2',
 'DK14_SKD_Deg_004.L1',
 'DK14_SKD_Deg_004.L1_1',
 'DK14_SKD_Deg_004.L1_2',
 'DK14_SKD_Deg_004.L2',
 'DK14_SKD_Deg_004.L2_1',
 'DK14_SKD_Deg_004.L2_2',
 'DK14_SKD_Deg_004.P1',
 'DK14_SKD_Deg_004.P2',
 'DK14_SKD_Deg_004.P2_1',
 'DK14_SKD_Deg_004.Q2',
 'DK14_SKD_Deg_004.Q2_2',
 'DK14_SKD_Deg_004.Q3_1_2',
 'DK14_SKD_Deg_004.Q3_2',
 'DK14_SKD_Deg_004.Q3_3',
 'DK14_SKD_Deg_004.Q4',
 'DK14_SKD_Deg_004.Q5',
 'DK14_SKD_Deg_004.Q6',
 'DK14_SKD_Deg_004.State1',
 'DK14_SKD_Deg_004.State2',
 'DK14_SKD_Deg_004.T1',
 'DK14_SKD_Deg_004.T2',
 'DK14_SKD_Deg_004.T3',
 'DK14_SKD_Deg_004.T4',
 '34апп.сумма.4нитка',
 'Суммарная подача пара на 4 нитку, т/час',
 '34апп.удел.сумма.4нитка',
 'удел.сумм.4нитка',
 '120апп.удел.4нитка']

nit5 = [ 'DK14_SKD_Deg_005.F1',
 'DK14_SKD_Deg_005.F1.8H',
 'DK14_SKD_Deg_005.F1_M',
 'DK14_SKD_Deg_005.F2_1',
 'DK14_SKD_Deg_005.F2_2',
 'DK14_SKD_Deg_005.F3_1',
 'DK14_SKD_Deg_005.F3_1.IsOverheat',
 'DK14_SKD_Deg_005.F3_1.T_sat',
 'DK14_SKD_Deg_005.F3_2',
 'DK14_SKD_Deg_005.F3_2.IsOverheat',
 'DK14_SKD_Deg_005.F3_2.T_sat',
 'DK14_SKD_Deg_005.F3_3',
 'DK14_SKD_Deg_005.F3_3.IsOverheat',
 'DK14_SKD_Deg_005.F3_3.T_sat',
 'DK14_SKD_Deg_005.F4',
 'DK14_SKD_Deg_005.F5',
 'DK14_SKD_Deg_005.F6',
 'DK14_SKD_Deg_005.F7_1',
 'DK14_SKD_Deg_005.F7_2',
 'DK14_SKD_Deg_005.FR3015',
 'DK14_SKD_Deg_005.FR3015.IsOverheat',
 'DK14_SKD_Deg_005.FR3015.T_sat',
 'DK14_SKD_Deg_005.FRC3015',
 'DK14_SKD_Deg_005.I1',
 'DK14_SKD_Deg_005.I2',#Ток мешалки?
 'DK14_SKD_Deg_005.L1',
 'DK14_SKD_Deg_005.L1_1',
 'DK14_SKD_Deg_005.L1_2',
 'DK14_SKD_Deg_005.L2',
 'DK14_SKD_Deg_005.L2_1',
 'DK14_SKD_Deg_005.L2_2',
 'DK14_SKD_Deg_005.P1',
 'DK14_SKD_Deg_005.P2',
 'DK14_SKD_Deg_005.P2_1',
 'DK14_SKD_Deg_005.Q2',
 'DK14_SKD_Deg_005.Q2_2',
 'DK14_SKD_Deg_005.Q3_1_2',
 'DK14_SKD_Deg_005.Q3_2',
 'DK14_SKD_Deg_005.Q3_3',
 'DK14_SKD_Deg_005.Q4',
 'DK14_SKD_Deg_005.Q5',
 'DK14_SKD_Deg_005.Q6',
 'DK14_SKD_Deg_005.State1',
 'DK14_SKD_Deg_005.State2',
 'DK14_SKD_Deg_005.T1',
 'DK14_SKD_Deg_005.T2',
 'DK14_SKD_Deg_005.T3',
 'DK14_SKD_Deg_005.T4',
 'DK14_SKD_Deg_005.TR103_1',
 'DK14_SKD_Deg_005.TR103_2',
 '34апп.сумма.5нитка',
 'Суммарная подача пара на 5 нитку, т/час',
 '34апп.удел.сумма.5нитка',
 'удел.сумм.5нитка',
 '120апп.удел.5нитка']
print(len(nit5), len(nit4), len(nit3))

55 52 52


In [40]:
#test
list_5 = ['DK14_SKD_Deg_005.L1_1','DK14_SKD_Deg_005.L1_2',
            'DK14_SKD_Deg_005.L2_1','DK14_SKD_Deg_005.L2_2',
            'DK14_SKD_Deg_005.Q2_2',
            'DK14_SKD_Deg_005.Q3_1_2','DK14_SKD_Deg_005.Q3_2',
            'DK14_SKD_Deg_005.Q3_3','DK14_SKD_Deg_005.Q4',
            'DK14_SKD_Deg_005.F1_M','DK14_SKD_Deg_005.F2_2',
            'DK14_SKD_Deg_005.F3_1.IsOverheat','DK14_SKD_Deg_005.F3_2.IsOverheat',
            'DK14_SKD_Deg_005.F4', 'DK14_SKD_Deg_005.F5',
            'DK14_SKD_Deg_005.F6', 'DK14_SKD_Deg_005.FR3015.IsOverheat',
            'DK14_SKD_Deg_005.I1', 'DK14_SKD_Deg_005.Q2',
            'DK14_SKD_Deg_005.Q5', 'DK14_SKD_Deg_005.Q6',
            'DK14_SKD_Deg_005.State1', 'DK14_SKD_Deg_005.State2',
            'DK14_SKD_Deg_005.TR103_1','DK14_SKD_Deg_005.TR103_2',
            'DK14_SKD_Deg_005.F7_1', 'DK14_SKD_Deg_005.F7_2','DK14_SKD_Deg_005.FRC3015', 'DK14_SKD_Deg_005.T3',# 57.9%, 100.0%, 57.9% missing values
            'DK14_SKD_Deg_005.F3_1.T_sat', 'DK14_SKD_Deg_005.F3_2.T_sat', 'DK14_SKD_Deg_005.FR3015.T_sat',# 6.9% missing values
            #'DK14_SKD_Deg_005.T1', 'DK14_SKD_Deg_005.T2',
            #'DK14_SKD_Deg_005.F3_3.IsOverheat', 'DK14_SKD_Deg_005.F3_3.T_sat'
         ]


list_4 = ['DK14_SKD_Deg_004.L1_1','DK14_SKD_Deg_004.L1_2','DK14_SKD_Deg_004.L2_1','DK14_SKD_Deg_004.L2_2',
          'DK14_SKD_Deg_004.Q2_2','DK14_SKD_Deg_004.Q3_1_2','DK14_SKD_Deg_004.Q3_2','DK14_SKD_Deg_004.Q3_3',
          'DK14_SKD_Deg_004.Q4','DK14_SKD_Deg_004.F7_1','DK14_SKD_Deg_004.F7_2',
          'DK14_SKD_Deg_004.F1_M','DK14_SKD_Deg_004.F2_2','DK14_SKD_Deg_004.F3_1.IsOverheat',
          'DK14_SKD_Deg_004.F3_2.IsOverheat','DK14_SKD_Deg_004.F3_3.IsOverheat','DK14_SKD_Deg_004.F4',
          'DK14_SKD_Deg_004.F5','DK14_SKD_Deg_004.F6','DK14_SKD_Deg_004.FR3012.IsOverheat',
          'DK14_SKD_Deg_004.FRC3012','DK14_SKD_Deg_004.I1','DK14_SKD_Deg_004.Q2',
          'DK14_SKD_Deg_004.Q5','DK14_SKD_Deg_004.Q6','DK14_SKD_Deg_004.State1',
          'DK14_SKD_Deg_004.State2',
          'DK14_SKD_Deg_004.F3_1.T_sat', 'DK14_SKD_Deg_004.F3_2.T_sat', 'DK14_SKD_Deg_004.FR3012.T_sat',#6.9%  missing values
          'DK14_SKD_Deg_004.F3_3.T_sat',
          #'DK14_SKD_Deg_004.T1', 'DK14_SKD_Deg_004.T2'
         ]

list_3 = ['DK14_SKD_Deg_003.L1_1','DK14_SKD_Deg_003.L1_2',
          'DK14_SKD_Deg_003.L2_1','DK14_SKD_Deg_003.L2_2',
          'DK14_SKD_Deg_003.Q2_2', 'DK14_SKD_Deg_003.I1',
          'DK14_SKD_Deg_003.Q3_1_2','DK14_SKD_Deg_003.Q3_2',
          'DK14_SKD_Deg_003.Q3_3','DK14_SKD_Deg_003.Q4',
          'DK14_SKD_Deg_003.F7_1','DK14_SKD_Deg_003.F7_2',
          'DK14_Shop.REZIM_3N','DK14_SKD_Deg_003.F1_M',
          'DK14_SKD_Deg_003.F2_2','DK14_SKD_Deg_003.F3_1.IsOverheat',
          'DK14_SKD_Deg_003.F3_2.IsOverheat','DK14_SKD_Deg_003.F4',
          'DK14_SKD_Deg_003.F5','DK14_SKD_Deg_003.F6',
          'DK14_SKD_Deg_003.FRC3009.IsOverheat','DK14_SKD_Deg_003.Q2',
          'DK14_SKD_Deg_003.Q5','DK14_SKD_Deg_003.Q6',
          'DK14_SKD_Deg_003.State1','DK14_SKD_Deg_003.State2',
          'DK14_SKD_Deg_003.TT100','DK14_SKD_Deg_003.TT101',
          'DK14_SKD_Deg_003.T3', 'DK14_SKD_Deg_003.T4']#72.2%, 55.1%  missing values


list_dop = ['DK14_SKD_Prod.QR1502',#27.9% miss
            'DK14_Shop.AR501_DK2', 'DK14_Shop.AR502_DK2', #100% miss
            #'VR010104EKSKD_TempH20.LHA', 'VR010104EKSTEAM_NORTH.LHA', 'VR010104EKSTEAM_SOUTH.LHA',#57.8% miss
            
           ]


# list_tmp_4_5 = list_4.copy()
# list_tmp_4_5.extend(list_5)
# list_tmp_4_5.extend(nit3)
# list_tmp_4_5.extend(list_dop)
# list_tmp_4_5.append('index')
# print('list_tmp_4_5', len(list_tmp_4_5))
list_tmp_4_5 = nit3.copy()#temp
df_4_5 = fin_df[(fin_df['DK14_SKD_Deg_005.F1'] > 0.1)\
              & (fin_df['DK14_SKD_Deg_004.F1'] > 0.1)\
              & (fin_df['Суммарная подача пара на 5 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.5нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.5нитка'] < 8)\
              & (fin_df['удел.сумм.5нитка'] < 10)\
              & (fin_df['Суммарная подача пара на 4 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.4нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.4нитка'] < 8)\
              & (fin_df['удел.сумм.4нитка'] < 10)\
              & (fin_df['DK14_SKD_Prod.Mn'] < 80)
              ].reset_index().drop(columns = list_tmp_4_5)
pandas_profiling.ProfileReport(df = df_4_5, minimal = True).to_file('df_4_5_full.html')#to_widgets()  

# list_tmp_3_5 = list_3.copy()
# list_tmp_3_5.extend(list_5)
# list_tmp_3_5.extend(nit4)
# list_tmp_3_5.extend(list_dop)
# list_tmp_3_5.append('index')
# print('list_tmp_3_5',len(list_tmp_3_5))

list_tmp_3_5 = nit4.copy()#temp
df_3_5 = fin_df[(fin_df['DK14_SKD_Deg_005.F1'] > 0.1)\
              & (fin_df['DK14_SKD_Deg_003.F1'] > 0.1)\
              & (fin_df['Суммарная подача пара на 5 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.5нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.5нитка'] < 8)\
              & (fin_df['удел.сумм.5нитка'] < 10)\
              & (fin_df['Суммарная подача пара на 3 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.3нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.3нитка'] < 8)\
              & (fin_df['удел.сумм.3нитка'] < 10)\
              & (fin_df['DK14_Shop.REZIM_3N'] == 0)\
              & (fin_df['DK14_SKD_Prod.Mn'] < 80)
              ].reset_index().drop(columns = list_tmp_3_5)
pandas_profiling.ProfileReport(df = df_3_5, minimal = True).to_file('df_3_5_full.html')#to_widgets()  

# list_tmp_3_4 = list_4.copy()
# list_tmp_3_4.extend(list_3)
# list_tmp_3_4.extend(nit5)
# list_tmp_3_4.extend(list_dop)
# list_tmp_3_4.append('index')
# print('list_tmp_3_4',len(list_tmp_3_4))

list_tmp_3_4 = nit5.copy()#temp
df_3_4  = fin_df[(fin_df['DK14_SKD_Deg_004.F1'] > 0.1)\
              & (fin_df['DK14_SKD_Deg_003.F1'] > 0.1)\
              & (fin_df['Суммарная подача пара на 4 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.4нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.4нитка'] < 8)\
              & (fin_df['удел.сумм.4нитка'] < 10)
              & (fin_df['Суммарная подача пара на 3 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.3нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.3нитка'] < 8)\
              & (fin_df['удел.сумм.3нитка'] < 10)\
              & (fin_df['DK14_Shop.REZIM_3N'] == 0)\
              & (fin_df['DK14_SKD_Deg_004.T1'] > 50)\
              & (fin_df['DK14_SKD_Prod.Mn'] < 80)
              ].reset_index().drop(columns = list_tmp_3_4)
pandas_profiling.ProfileReport(df = df_3_4, minimal = True).to_file('df_3_4_full.html')#to_widgets()  

list_tmp_3_4_5 = list_tmp_4_5.copy()
list_tmp_3_4_5.extend(list_3)
print('list_tmp_3_4_5',len(list_tmp_3_4_5))
df_3_4_5  = fin_df[(fin_df['DK14_SKD_Deg_004.F1'] > 0.1)\
              & (fin_df['DK14_SKD_Deg_003.F1'] > 0.1)\
              & (fin_df['DK14_SKD_Deg_005.F1'] > 0.1)\
              & (fin_df['Суммарная подача пара на 4 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.4нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.4нитка'] < 8)\
              & (fin_df['удел.сумм.4нитка'] < 10)
              & (fin_df['Суммарная подача пара на 3 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.3нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.3нитка'] < 8)\
              & (fin_df['удел.сумм.3нитка'] < 10)\
              & (fin_df['DK14_Shop.REZIM_3N'] == 0)\
              & (fin_df['Суммарная подача пара на 5 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.5нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.5нитка'] < 8)\
              & (fin_df['удел.сумм.5нитка'] < 10)\
              & (fin_df['DK14_SKD_Prod.Mn'] < 80)
              ].reset_index().drop(columns = list_tmp_3_4_5)
#pandas_profiling.ProfileReport(df = df_3_4_5, minimal = True).to_file('df_3_4_5.html')#to_widgets()  
print('df_3_4:',df_3_4.shape)
print('df_3_5:',df_3_5.shape)
print('df_4_5:',df_4_5.shape)
print('df_3_4_5:',df_3_4_5.shape)


list_tmp_3_4_5 82
df_3_4: (6391, 120)
df_3_5: (7447, 123)
df_4_5: (30192, 123)
df_3_4_5: (0, 123)


In [80]:
# nitkas[0].columns.to_list()
# r = {'DK14_SKD_Deg_005.T1':'34 дег., температура',
#  'DK14_SKD_Deg_005.T2':'120 дег., температура',
#  'DK14_SKD_Deg_005.P1':'34 дег. давление',
#  'DK14_SKD_Deg_005.P2':'120 дег. давление',
#  'DK14_SKD_Deg_005.F1':'34 дег. подача полим, т\ч',
#  'VR010104EKSTEAM_NORTH.LHA':'пар СЕВЕР, давление',
#  'VR010104EKSTEAM_SOUTH.LHA':'пар ЮГ, давление',
#  'DK14_SKD_Prod.QR1502':'pH цирк.воды',
#  'DK14_SKD_Prod.QR1501':'pH H-651-3',
#  'DK14_SKD_Prod.Mn':'Муни',
#  'DK14_SKD_Prod.SO':'Сухой остаток'}
# top=0.98,
# bottom=0.255,
# left=0.18,
# right=0.9,
# hspace=0.2,
# wspace=0.2
#sns.heatmap(, annot = True)
t123 = df_3_4.reset_index().drop(columns= ['Дата и время', 'index']).corr()
sns.heatmap(t123)
t123[\
    (t123[t123.columns.to_list()] > 0.5)\
    |(t123[t123.columns.to_list()] < -0.5)]

,df_index,120апп.удел.3нитка,120апп.удел.4нитка,34апп.сумма.3нитка,34апп.сумма.4нитка,34апп.удел.сумма.3нитка,34апп.удел.сумма.4нитка,DK14_SKD_Deg_003.F1,DK14_SKD_Deg_003.F1_M,DK14_SKD_Deg_003.F2_1,...,VR010104EKSTEAM_NORTH.LHA,VR010104EKSTEAM_SOUTH.LHA,"Массовая доля тримеров, %","Массовая доля углеводородов, %","Суммарная подача пара на 3 нитку, т/час","Суммарная подача пара на 4 нитку, т/час",удел.сумм.3нитка,удел.сумм.4нитка,Муни,Площадь релаксации
df_index,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120апп.удел.3нитка,NaN,1.000000,0.841983,-0.723582,-0.652355,0.899608,0.719137,-0.953445,-0.931561,-0.638870,...,NaN,NaN,NaN,NaN,-0.674026,-0.610584,0.970154,0.799159,NaN,NaN
120апп.удел.4нитка,NaN,0.841983,1.000000,-0.705391,-0.721753,0.713398,0.879728,-0.884907,-0.872045,-0.628277,...,NaN,NaN,NaN,NaN,-0.651696,-0.660983,0.792389,0.963558,NaN,NaN
34апп.сумма.3нитка,NaN,-0.723582,-0.705391,1.000000,0.918578,NaN,NaN,0.801335,0.788275,0.695459,...,NaN,NaN,NaN,NaN,0.984007,0.907676,-0.559105,-0.554144,NaN,NaN
34апп.сумма.4нитка,NaN,-0.652355,-0.721753,0.918578,1.000000,NaN,NaN,0.758580,0.747309,0.671127,...,NaN,NaN,NaN,NaN,0.899799,0.982357,-0.502937,-0.540178,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Суммарная подача пара на 4 нитку, т/час",NaN,-0.610584,-0.660983,0.907676,0.982357,NaN,NaN,0.740766,0.731329,0.660058,...,NaN,NaN,NaN,NaN,0.913059,1.000000,NaN,NaN,NaN,NaN
удел.сумм.3нитка,NaN,0.970154,0.792389,-0.559105,-0.502937,0.978653,0.761093,-0.916507,-0.894355,-0.562923,...,NaN,NaN,NaN,NaN,-0.523633,NaN,1.000000,0.799522,NaN,NaN
удел.сумм.4нитка,NaN,0.799159,0.963558,-0.554144,-0.540178,0.762539,0.974859,-0.838147,-0.827748,-0.553192,...,NaN,NaN,NaN,NaN,-0.511863,NaN,0.799522,1.000000,NaN,NaN
Муни,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [ ]:
df_4  = fin_df[['DK14_SKD_Deg_004.P1','Дата и время','Массовая доля углеводородов, %']][(fin_df['DK14_SKD_Deg_004.P1'] > 0.2)\
               & (fin_df['DK14_SKD_Deg_004.P1'] < 0.3)\
               & (fin_df['Дата и время'] > datetime(2019,6,3,0,0))\
               & (fin_df['Дата и время'] < datetime(2019,6,5,0,0))\
#               & (fin_df['120апп.удел.4нитка'] < 5)\
#               & (fin_df['34апп.удел.сумма.4нитка'] < 8)\
#               & (fin_df['удел.сумм.4нитка'] < 10)
              ]#.reset_index()
pd.set_option("display.max_rows", None, "display.max_columns", None)
print(df_4)
#df_4['DK14_SKD_Deg_004.P1'].plot.kde()
#mn.ppk_plot(df_4, lower_control_limit=0.2, upper_control_limit=0.5)

In [ ]:
print(df_4_5.columns.to_list())
df45 = df_4_5[['удел.сумм.4нитка','удел.сумм.5нитка','Массовая доля углеводородов, %',
         'Дата и время', 'DK14_SKD_Deg_004.T1', 'DK14_SKD_Deg_004.T2',
         'DK14_SKD_Deg_005.T1', 'DK14_SKD_Deg_005.T2',
          'VR010104EKSTEAM_NORTH.LHA','VR010104EKSTEAM_SOUTH.LHA',
               'DK14_SKD_Deg_005.P1', 'DK14_SKD_Deg_005.P2',
               'DK14_SKD_Deg_004.P1', 'DK14_SKD_Deg_004.P2',
               'Суммарная подача пара на 4 нитку, т/час', 'Суммарная подача пара на 5 нитку, т/час',
               'DK14_SKD_Deg_004.F1','DK14_SKD_Deg_005.F1'
               
        ]][(df_4_5['Дата и время']> datetime(2020,1,1))].reset_index().drop(columns = 'index').sort_values(by = 'Дата и время')
#df45.to_excel('tmp.xlsx')
print(df45.describe(), df45.median())
print('---------------------------------------------------------------------------------------')
tmp45 = df45[
             
              #df45['удел.сумм.4нитка'] < 4.5)\
             #&(df45['удел.сумм.5нитка'] < 4.5)\
             (df45['DK14_SKD_Deg_004.P2'] > 0.7)]
print(tmp45.describe(),tmp45.median())
sns.lineplot(data=df45, x="Дата и время", y="DK14_SKD_Deg_004.T1")
sns.lineplot(data=df45, x="Дата и время", y="DK14_SKD_Deg_004.T2")
sns.lineplot(data=df45, x="Дата и время", y="удел.сумм.4нитка")
sns.lineplot(data=df45, x="Дата и время", y="Массовая доля углеводородов, %")
#print(df45.drop(columns = 'Дата и время').astype(float).corr())
#pandas_profiling.ProfileReport(df = df45).to_file('df_4_5.html')

In [ ]:
rename_4_5_dict = {'120апп.удел.4нитка':'120апп.удел_x',\
 '120апп.удел.5нитка':'120апп.удел_y',
 '34апп.сумма.4нитка':'34апп.сумма_x',
 '34апп.сумма.5нитка':'34апп.сумма_y',
 '34апп.удел.сумма.4нитка':'34апп.удел.сумма_x',
 '34апп.удел.сумма.5нитка':'34апп.удел.сумма_y',
 'DK14_SKD_Deg_004.F1': 'F1_x',
 'DK14_SKD_Deg_004.F2_1':'F2_1_x',
 'DK14_SKD_Deg_004.F3_1':'F3_1_x',
 'DK14_SKD_Deg_004.F3_1.T_sat': 'F3_1.T_sat_x',
 'DK14_SKD_Deg_004.F3_2': 'F3_2_x',
 'DK14_SKD_Deg_004.F3_2.T_sat': 'F3_2.T_sat_x',
 'DK14_SKD_Deg_004.F3_3': 'F3_3_x',
 'DK14_SKD_Deg_004.F3_3.T_sat': 'F3_3.T_sat_x',
 'DK14_SKD_Deg_004.FR3012': 'FR30_x',
 'DK14_SKD_Deg_004.FR3012.T_sat':'FR30.T_sat_x',
 'DK14_SKD_Deg_004.I2':'I2_x',
 'DK14_SKD_Deg_004.L1': 'L1_x',
 'DK14_SKD_Deg_004.L2':'L2_x',
 'DK14_SKD_Deg_004.P1':'P1_x',
 'DK14_SKD_Deg_004.P2':'P2_x',
 'DK14_SKD_Deg_004.P2_1':'P2_1_x',
 'DK14_SKD_Deg_004.T3':'T3_x',
 'DK14_SKD_Deg_004.T4':'T4_x',
 'DK14_SKD_Deg_005.F1':'F1_y',
 'DK14_SKD_Deg_005.F1.8H':'F1.8H_y',
 'DK14_SKD_Deg_005.F2_1':'F2_1_y',
 'DK14_SKD_Deg_005.F3_1':'F3_1_y',
 'DK14_SKD_Deg_005.F3_2':'F3_2_y',
 'DK14_SKD_Deg_005.F3_3':'F3_3_y',
 'DK14_SKD_Deg_005.F3_3.IsOverheat':'F3_3.IsOverheat_y',
 'DK14_SKD_Deg_005.F3_3.T_sat':'F3_3.T_sat_y',
 'DK14_SKD_Deg_005.FR3015':'FR30_y',
 'DK14_SKD_Deg_005.I2':'I2_y',
 'DK14_SKD_Deg_005.L1':'L1_y',
 'DK14_SKD_Deg_005.L2':'L2_y',
 'DK14_SKD_Deg_005.P1':'P1_y',
 'DK14_SKD_Deg_005.P2':'P2_y',
 'DK14_SKD_Deg_005.P2_1':'P2_1_y',
 'DK14_SKD_Deg_005.T4':'T4_y',
 'Суммарная подача пара на 4 нитку, т/час':'Суммарная подача пара_x',
 'Суммарная подача пара на 5 нитку, т/час':'Суммарная подача пара_y',
 'удел.сумм.4нитка':'удел.сумм_x',
 'удел.сумм.5нитка':'удел.сумм_y'}

rename_3_5_dict = {'120апп.удел.3нитка':'120апп.удел_x',\
 '120апп.удел.5нитка':'120апп.удел_y',
 '34апп.сумма.3нитка':'34апп.сумма_x',
 '34апп.сумма.5нитка':'34апп.сумма_y',
 '34апп.удел.сумма.3нитка':'34апп.удел.сумма_x',
 '34апп.удел.сумма.5нитка':'34апп.удел.сумма_y',
 'DK14_SKD_Deg_003.F1': 'F1_x',
 'DK14_SKD_Deg_003.F2_1':'F2_1_x',
 'DK14_SKD_Deg_003.F3_1':'F3_1_x',
 'DK14_SKD_Deg_003.F3_1.T_sat': 'F3_1.T_sat_x',
 'DK14_SKD_Deg_003.F3_2': 'F3_2_x',
 'DK14_SKD_Deg_003.F3_2.T_sat': 'F3_2.T_sat_x',
 'DK14_SKD_Deg_003.F3_3': 'F3_3_x',
 'DK14_SKD_Deg_003.F3_3.T_sat': 'F3_3.T_sat_x',
 'DK14_SKD_Deg_003.FRC3009': 'FR30_x',
 'DK14_SKD_Deg_003.FRC3009.T_sat':'FR30.T_sat_x',
 'DK14_SKD_Deg_003.I2':'I2_x',
 'DK14_SKD_Deg_003.L1': 'L1_x',
 'DK14_SKD_Deg_003.L2':'L2_x',
 'DK14_SKD_Deg_003.P1':'P1_x',
 'DK14_SKD_Deg_003.P2':'P2_x',
 'DK14_SKD_Deg_003.P2_1':'P2_1_x',
 'DK14_SKD_Deg_003.T3':'T3_x',
 'DK14_SKD_Deg_003.T4':'T4_x',
 'DK14_SKD_Deg_005.F1':'F1_y',
 'DK14_SKD_Deg_005.F1.8H':'F1.8H_y',
 'DK14_SKD_Deg_005.F2_1':'F2_1_y',
 'DK14_SKD_Deg_005.F3_1':'F3_1_y',
 'DK14_SKD_Deg_005.F3_2':'F3_2_y',
 'DK14_SKD_Deg_005.F3_3':'F3_3_y',
 'DK14_SKD_Deg_005.F3_3.IsOverheat':'F3_3.IsOverheat_y',
 'DK14_SKD_Deg_005.F3_3.T_sat':'F3_3.T_sat_y',
 'DK14_SKD_Deg_005.FR3015':'FR30_y',
 'DK14_SKD_Deg_005.I2':'I2_y',
 'DK14_SKD_Deg_005.L1':'L1_y',
 'DK14_SKD_Deg_005.L2':'L2_y',
 'DK14_SKD_Deg_005.P1':'P1_y',
 'DK14_SKD_Deg_005.P2':'P2_y',
 'DK14_SKD_Deg_005.P2_1':'P2_1_y',
 'DK14_SKD_Deg_005.T4':'T4_y',
 'Суммарная подача пара на 3 нитку, т/час':'Суммарная подача пара_x',
 'Суммарная подача пара на 5 нитку, т/час':'Суммарная подача пара_y',
 'удел.сумм.3нитка':'удел.сумм_x',
 'удел.сумм.5нитка':'удел.сумм_y'}

rename_3_4_dict = {'120апп.удел.3нитка':'120апп.удел_x',\
 '120апп.удел.4нитка':'120апп.удел_y',
 '34апп.сумма.3нитка':'34апп.сумма_x',
 '34апп.сумма.4нитка':'34апп.сумма_y',
 '34апп.удел.сумма.3нитка':'34апп.удел.сумма_x',
 '34апп.удел.сумма.4нитка':'34апп.удел.сумма_y',
 'DK14_SKD_Deg_003.F1': 'F1_x',
 'DK14_SKD_Deg_003.F2_1':'F2_1_x',
 'DK14_SKD_Deg_003.F3_1':'F3_1_x',
 'DK14_SKD_Deg_003.F3_1.T_sat': 'F3_1.T_sat_x',
 'DK14_SKD_Deg_003.F3_2': 'F3_2_x',
 'DK14_SKD_Deg_003.F3_2.T_sat': 'F3_2.T_sat_x',
 'DK14_SKD_Deg_003.F3_3': 'F3_3_x',
 'DK14_SKD_Deg_003.F3_3.T_sat': 'F3_3.T_sat_x',
 'DK14_SKD_Deg_003.FRC3009': 'FR30_x',
 'DK14_SKD_Deg_003.FRC3009.T_sat':'FR30.T_sat_x',
 'DK14_SKD_Deg_003.I2':'I2_x',
 'DK14_SKD_Deg_003.L1': 'L1_x',
 'DK14_SKD_Deg_003.L2':'L2_x',
 'DK14_SKD_Deg_003.P1':'P1_x',
 'DK14_SKD_Deg_003.P2':'P2_x',
 'DK14_SKD_Deg_003.P2_1':'P2_1_x',
 'DK14_SKD_Deg_003.T3':'T3_x',
 'DK14_SKD_Deg_003.T4':'T4_x',
 'DK14_SKD_Deg_004.F1':'F1_y',
 'DK14_SKD_Deg_004.F1.8H':'F1.8H_y',
 'DK14_SKD_Deg_004.F2_1':'F2_1_y',
 'DK14_SKD_Deg_004.F3_1':'F3_1_y',
 'DK14_SKD_Deg_004.F3_2':'F3_2_y',
 'DK14_SKD_Deg_004.F3_3':'F3_3_y',
 'DK14_SKD_Deg_004.F3_3.IsOverheat':'F3_3.IsOverheat_y',
 'DK14_SKD_Deg_004.F3_3.T_sat':'F3_3.T_sat_y',
 'DK14_SKD_Deg_004.FR3012':'FR30_y',
 'DK14_SKD_Deg_004.I2':'I2_y',
 'DK14_SKD_Deg_004.L1':'L1_y',
 'DK14_SKD_Deg_004.L2':'L2_y',
 'DK14_SKD_Deg_004.P1':'P1_y',
 'DK14_SKD_Deg_004.P2':'P2_y',
 'DK14_SKD_Deg_004.P2_1':'P2_1_y',
 'DK14_SKD_Deg_004.T3':'T3_y',
 'DK14_SKD_Deg_004.T4':'T4_y',
 'Суммарная подача пара на 3 нитку, т/час':'Суммарная подача пара_x',
 'Суммарная подача пара на 4 нитку, т/час':'Суммарная подача пара_y',
 'удел.сумм.3нитка':'удел.сумм_x',
 'удел.сумм.4нитка':'удел.сумм_y'}

In [ ]:
#TODO - CHECK T1 T2
df_4_5[np.isnan(df_4_5['Массовая доля углеводородов, %']) == False]

In [ ]:
#1136  - всего анализов на органику
#528 от df_4_5
#149 от df_3_4
#184 от 3_5
#Подготовка df (MAIN CELL)
number_of_organic_tests = fin_df[np.isnan(fin_df['Массовая доля углеводородов, %']) == False].shape[0]
print('всего анализов на органику:', number_of_organic_tests)

#df_4_5 = df_4_5.drop(columns = tmp_list)

dfs = []
dfs.append(df_4_5[np.isnan(df_4_5['Массовая доля углеводородов, %']) == False]\
                                                .reset_index()\
                                                .drop(columns = ['Массовая доля тримеров, %', 'Дата и время', 'index'])\
                                                .dropna()\
                                                .astype(float))
dfs.append(df_3_4[np.isnan(df_3_4['Массовая доля углеводородов, %']) == False]\
                                                .reset_index()\
                                                .drop(columns = ['Массовая доля тримеров, %', 'Дата и время', 'index'])\
                                                .dropna()\
                                                .astype(float))
dfs.append(df_3_5[np.isnan(df_3_5['Массовая доля углеводородов, %']) == False]\
                                                .reset_index()\
                                                .drop(columns = ['Массовая доля тримеров, %', 'Дата и время', 'index'])\
                                                .dropna()\
                                                .astype(float))

losing_of_tests = number_of_organic_tests
for i in dfs:
    print(i.shape)
    losing_of_tests -= i.shape[0]
print('Потеряно анализов на органику:', losing_of_tests)

In [ ]:
#TEST CELL append'a линий
number_of_organic_tests = fin_df[np.isnan(fin_df['Массовая доля углеводородов, %']) == False].shape[0]
print('всего анализов на органику:', number_of_organic_tests)

df_4_5_tmp = df_4_5.rename(columns = rename_4_5_dict)
df_3_4_tmp = df_3_4.rename(columns = rename_3_4_dict).drop(columns=['DK14_SKD_Deg_003.T1','DK14_SKD_Deg_003.T2'])
df_3_5_tmp = df_3_5.rename(columns = rename_3_5_dict).drop(columns=['DK14_SKD_Deg_003.T1','DK14_SKD_Deg_003.T2'])

dfs = []
dfs.append(df_4_5_tmp[np.isnan(df_4_5_tmp['Массовая доля углеводородов, %']) == False]\
                                                .reset_index()\
                                                .drop(columns = ['Массовая доля тримеров, %', 'Дата и время', 'index'])\
                                                .dropna()\
                                                .astype(float))
dfs.append(df_3_4_tmp[np.isnan(df_3_4_tmp['Массовая доля углеводородов, %']) == False]\
                                                .reset_index()\
                                                .drop(columns = ['Массовая доля тримеров, %', 'Дата и время', 'index'])\
                                                .dropna()\
                                                .astype(float))
dfs.append(df_3_5_tmp[np.isnan(df_3_5_tmp['Массовая доля углеводородов, %']) == False]\
                                                .reset_index()\
                                                .drop(columns = ['Массовая доля тримеров, %', 'Дата и время', 'index'])\
                                                .dropna()\
                                                .astype(float))
sum_df_tmp = dfs[0].append(dfs[1])
sum_df_tmp = sum_df_tmp.append(dfs[2])
dfs.append(sum_df_tmp.dropna(axis = 1))


losing_of_tests = number_of_organic_tests
for i in dfs:
    print(i.shape)
    losing_of_tests -= i.shape[0]
print('Потеряно анализов на органику:', losing_of_tests)
'TEST_tmp'

In [ ]:
#Данные для обучения и теста модели
#При объеме данных менее 400 наблюдений
#линейная регрессия не способна обучиться чему-либо
scaler = MinMaxScaler()#StandardScaler()
tmp_df = dfs[-1]

scaler_for_neural = StandardScaler()
arr_for_neural = scaler_for_neural.fit_transform(tmp_df)
X_for_neural = np.delete(arr_for_neural, tmp_df.columns.to_list().index('Массовая доля углеводородов, %'), 1)
y_for_neural = arr_for_neural[:,tmp_df.columns.to_list().index('Массовая доля углеводородов, %')]
X_train_neural, X_test_neural, y_train_neural, y_test_neural = train_test_split(X_for_neural, y_for_neural, random_state=20)

n_arr_norm = scaler.fit_transform(tmp_df)
print('Размер данных=', tmp_df.shape)
#X = tmp_df.drop(columns = ['Массовая доля углеводородов, %'])
#y = tmp_df['Массовая доля углеводородов, %']
X = np.delete(n_arr_norm, tmp_df.columns.to_list().index('Массовая доля углеводородов, %'), 1)
y = n_arr_norm[:,tmp_df.columns.to_list().index('Массовая доля углеводородов, %')]
#X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(), y.to_numpy(), random_state=20)#, test_size = 0.2, train_size = 0.8)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20)#, test_size = 0.2, train_size = 0.8)
print('Обучающие датасеты', X_train, y_train)
print('Размер Х_train', len(X_train))
print('Размер Х_test', len(X_test))

In [ ]:
# Коэффициент детерминации R-квадрат для модели с константой принимает значения от 0 до 1.
# Чем ближе значение коэффициента к 1, тем сильнее зависимость.
# При оценке регрессионных моделей это интерпретируется как соответствие модели данным.
# Для приемлемых моделей предполагается, что коэффициент детерминации должен быть хотя бы не меньше 50 %
# (в этом случае коэффициент множественной корреляции превышает по модулю 70 %). 
# Модели с коэффициентом детерминации выше 80 % можно признать достаточно хорошими (коэффициент корреляции превышает 90 %).
# Значение коэффициента детерминации 1 означает функциональную зависимость между переменными.
print('Линейная регрессия, метод наименьших квадратов:')
lr = LinearRegression().fit(X_train, y_train)
print("lr.coef_: {}".format(lr.coef_))
print("lr.intercept_: {}".format(lr.intercept_))
print("R-квадрат на обучающем наборе: {:.2f}".format(lr.score(X_train, y_train)))
print("R-квадрат на тестовом наборе: {:.2f}".format(lr.score(X_test, y_test)))
print('mse:', mean_squared_error(y_test, lr.predict(X_test)))
#lr.predict(X_test)

In [ ]:
#Гребневая регрессия(L2 регуляризация[alpha])
#Оптимальное значение alpha зависит от конкретного используемого набора данных. 
#Увеличение alpha заставляет коэффициенты сжиматься до близких к нулю значений,
#что снижает качество работы модели на обучающем наборе, но может улучшить ее обобщающую способность.
#best params {'alpha': 1}
#best score 0.4651694547672382
parameters = {'alpha':range(200)}
ridge = Ridge()
ridge_grid = GridSearchCV(ridge, parameters,
cv=5, n_jobs=-1,
verbose=True)
ridge_grid.fit(X_train, y_train)
print('best params',ridge_grid.best_params_)
print('best score',ridge_grid.best_score_)
print('Гребневая регрессия(L2 регуляризация[alpha])')
for alpha in [0, 0.0001, 0.0005,0.001, 0.01, 0.02, 0.005, 0.0075, 0.1, 0.3, 0.5, 1, 3, 10, 15, 50, 74, 100]:
    ridge = Ridge(alpha = alpha).fit(X_train, y_train)
    print('alpha =', alpha )
    print("R-квадрат на обучающем наборе: {:.2f}".format(ridge.score(X_train, y_train)))
    print("R-квадрат на тестовом наборе: {:.2f}".format(ridge.score(X_test, y_test)))
    print('mse:', mean_squared_error(y_test, ridge.predict(X_test)))


In [ ]:
#Лассо (L1 регуляризация[alpha])
#Результат L1 регуляризации заключается в том,
#что при использовании лассо некоторые коэффициенты становятся равны точно нулю.
#best params {'alpha': 0}
#best score 0.4561862495198607
parameters = {'alpha':range(200)}
lasso = Lasso()
lasso_grid = GridSearchCV(lasso, parameters,
cv=5, n_jobs=-1,
verbose=True)
lasso_grid.fit(X_train, y_train)
print('best params',lasso_grid.best_params_)
print('best score',lasso_grid.best_score_)
for alpha in [0, 0.0001, 0.0005,0.001, 0.01, 0.02, 0.005, 0.0075, 0.1, 0.3, 0.5, 1, 3, 10, 15, 50, 100]:
    lasso = Lasso(alpha = alpha).fit(X_train, y_train)
    print("R-квадрат на обучающем наборе: {:.2f}".format(lasso.score(X_train, y_train)))
    print("R-квадрат на контрольном наборе: {:.2f}".format(lasso.score(X_test, y_test)))
    print('mse:', mean_squared_error(y_test, lasso.predict(X_test)))
    print("Количество использованных признаков: {}".format(np.sum(lasso.coef_ != 0)))
    print('################################################')
#print(lasso.coef_)

In [ ]:
parameters = {'max_depth' : range(10)}
tree = DecisionTreeRegressor(random_state = 0)
tree_grid = GridSearchCV(tree, parameters,
cv=5, n_jobs=-1,
verbose=True)
tree_grid.fit(X_train, y_train)
print('best params',tree_grid.best_params_)
print('best score',tree_grid.best_score_)
tree = DecisionTreeRegressor(random_state=0, max_depth = 4)
tree.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(tree.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(tree.score(X_test, y_test)))
print('mse:', mean_squared_error(y_test, tree.predict(X_test)))
print("Важности признаков:\n{}".format(tree.feature_importances_))

In [ ]:
#Случайный лес
#best params {'max_depth': 4, 'max_features': 22, 'n_estimators': 100}
#best score 0.5279822164083109
#4.2min
#-----------------------
#RandomForestRegressor(n_estimators=200, random_state=33,max_features = 38)
#Правильность на обучающем наборе: 0.937
#Правильность на тестовом наборе: 0.564

parameters = {'max_depth' : range(5), 'n_estimators': [50,100,150,200],'max_features':range(38)}

# forest = RandomForestRegressor(random_state = 0)
# forest_grid = GridSearchCV(forest, parameters,
# cv=5, n_jobs=-1,
# verbose=True)
# forest_grid.fit(X_train, y_train)
# print('best params',forest_grid.best_params_)
# print('best score',forest_grid.best_score_)
forest = RandomForestRegressor(n_estimators=100, random_state=33,max_features = 22, max_depth = 4)
forest.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(forest.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(forest.score(X_test, y_test)))
print('mse:', mean_squared_error(y_test, forest.predict(X_test)))

In [ ]:
#Градиентный бустинг
#best params {'learning_rate': 0.1, 'max_depth': 4, 'max_features': 14, 'n_estimators': 200}
#best score 0.5539923066393246
#33.8min
parameters = {'max_depth' : range(5), 'n_estimators': [50,100,150,200],'max_features':range(38),\
              'learning_rate':[0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1]}

#gbr = GradientBoostingRegressor(random_state = 11)
#gbr_grid = GridSearchCV(gbr, parameters,
#cv=5, n_jobs=-1,
#verbose=True)
#gbr_grid.fit(X_train, y_train)
#print('best params',gbr_grid.best_params_)
#print('best score',gbr_grid.best_score_)


#for i in [0.0001, 0.0005,0.001, 0.01, 0.02, 0.05, 0.075, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 1]:
#gbr_grid.best_params_['learning_rate']
gbrt = GradientBoostingRegressor(random_state=11, learning_rate=0.1, max_depth=4, max_features=14, n_estimators=200)

gbrt.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(gbrt.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(gbrt.score(X_test, y_test)))
print('mse:', mean_squared_error(y_test, gbrt.predict(X_test)))

In [ ]:
#AdaBoostRegressor
# best params {'base_estimator': DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=4,
#                       max_features=None, max_leaf_nodes=None,
#                       min_impurity_decrease=0.0, min_impurity_split=None,
#                       min_samples_leaf=1, min_samples_split=2,
#                       min_weight_fraction_leaf=0.0, presort='deprecated',
#                       random_state=0, splitter='best'), 'learning_rate': 0.01, 'n_estimators': 200}
# best score 0.526530481834167
parameters = {'n_estimators': [50,100,150,200],'learning_rate':[0, 0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1],\
              'base_estimator':[lr, tree, lasso, ridge]}    
# ada = AdaBoostRegressor(random_state=33)
# ada_grid = GridSearchCV(ada, parameters,
# cv=5, n_jobs=-1,
# verbose=True)
# ada_grid.fit(X_train, y_train)
# print('best params',ada_grid.best_params_)
# print('best score',ada_grid.best_score_)
ada = AdaBoostRegressor(random_state=33, learning_rate=0.01, base_estimator = tree, n_estimators=200)

ada.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(ada.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(ada.score(X_test, y_test)))
print('mse:', mean_squared_error(y_test, ada.predict(X_test)))

In [ ]:
#RandomForestRegressor
# best params {'max_depth': 4, 'max_features': 26, 'min_samples_split': 4, 'n_estimators': 50}
# best score 0.5265164417396928
# 26.0min finished
parameters = {'max_depth':range(5), 'n_estimators':[50,100,150,200],'max_features':range(38), 'min_samples_split':range(5)}
# rfr = RandomForestRegressor(random_state = 33, n_jobs = -1)
# rfr_grid = GridSearchCV(rfr, parameters,
# cv=5, n_jobs=-1,
# verbose=True)
# rfr_grid.fit(X_train, y_train)
# print('best params',rfr_grid.best_params_)
# print('best score',rfr_grid.best_score_)

rfr = RandomForestRegressor(n_jobs = -1,random_state=33, min_samples_split = 4,\
                            max_features = 26, n_estimators=50, max_depth = 4)
rfr.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(rfr.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(rfr.score(X_test, y_test)))
print('mse:', mean_squared_error(y_test, rfr.predict(X_test)))

In [ ]:
#VotingRegressor
# best params {'estimators':[('ada',ada),('gb',gbrt)]}
# best score 0.5636507180594125
#31.1s

# parameters = {'estimators':[\
#                             [('lr',lr),('dt',tree),('lasso',lasso), ('ridge',ridge),('rf',rfr), ('ada',ada),('gb',gbrt)],\
#                             [('rf',rfr), ('ada',ada),('gb',gbrt)],\
#                             [('lr',lr),('rf',rfr), ('ada',ada),('gb',gbrt)],\
#                             [('dt',tree),('rf',rfr), ('ada',ada),('gb',gbrt)],\
#                             [('lr',lr),('dt',tree),('lasso',lasso), ('ridge',ridge)],\
#                             [('ada',ada),('gb',gbrt)],\
#                             [('ada',ada),('rf',rfr)],\
#                             [('rfr',rfr),('gb',gbrt)]\
#                            ]}
# vr = VotingRegressor(())
# vr_grid = GridSearchCV(vr, parameters,
# cv=5, n_jobs=-1,
# verbose=True)
# vr_grid.fit(X_train, y_train)
# print('best params',vr_grid.best_params_)
# print('best score',vr_grid.best_score_)



#(lr,dt,tree,lasso,ridge,rfr,ada,gbrt)
    #Правильность на обучающем наборе: 0.650
    #Правильность на тестовом наборе: 0.505
#(rfr,ada,gbrt)
    #Правильность на обучающем наборе: 0.796
    #Правильность на тестовом наборе: 0.555
#(lr,rfr,ada,gbrt)
    #Правильность на обучающем наборе: 0.750
    #Правильность на тестовом наборе: 0.554
#(dt,rfr,ada,gbrt)
    #Правильность на обучающем наборе: 0.760
    #Правильность на тестовом наборе: 0.544
#(lr,dt,lasso,ridge)
    #Правильность на обучающем наборе: 0.478
    #Правильность на тестовом наборе: 0.424
#(ada, gb)
#Правильность на обучающем наборе: 0.858
#Правильность на тестовом наборе: 0.556
vr = VotingRegressor([\
                      #('lr',lr),\
                      #('dt',tree),\
                      #('lasso',lasso), ('ridge',ridge),\
                      #('rf',rfr),\
                      ('ada',ada),\
                      ('gb',gbrt)\
                     ])
vr.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(vr.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(vr.score(X_test, y_test)))
print('mse:', mean_squared_error(y_test, vr.predict(X_test)))


In [ ]:
#StackingRegressor
# best params {'estimators':  [('lr',lr),('dt',tree),('lasso',lasso), ('ridge',ridge),('rf',rfr), ('ada',ada),('gb',gbrt)],
#'final_estimator': ada
# best score 0.5548101391714823
# 5.5min finished
# parameters = {'estimators':[\
#                             [('lr',lr),('dt',tree),('lasso',lasso), ('ridge',ridge),('rf',rfr), ('ada',ada),('gb',gbrt)],\
#                             [('rf',rfr), ('ada',ada),('gb',gbrt)],\
#                             [('lr',lr),('rf',rfr), ('ada',ada),('gb',gbrt)],\
#                             [('dt',tree),('rf',rfr), ('ada',ada),('gb',gbrt)],\
#                             [('lr',lr),('dt',tree),('lasso',lasso), ('ridge',ridge)],\
#                             [('ada',ada),('gb',gbrt)],\
#                             [('ada',ada),('rf',rfr)],\
#                             [('rfr',rfr),('gb',gbrt)]\
#                            ],\
#              'final_estimator':[ada,rfr,gbrt]}
# sr = StackingRegressor(())
# sr_grid = GridSearchCV(sr, parameters,
# cv=5, n_jobs=-1,
# verbose=True)
# sr_grid.fit(X_train, y_train)
# print('best params',sr_grid.best_params_)
# print('best score',sr_grid.best_score_)

sr = StackingRegressor(estimators = [\
                      ('lr',lr),\
                      ('dt',tree),\
                      ('lasso',lasso), ('ridge',ridge),\
                      ('rf',rfr),\
                      ('ada',ada),\
                      ('gb',gbrt)\
                     ], final_estimator = ada)
sr.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(sr.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(sr.score(X_test, y_test)))
print('mse:', mean_squared_error(y_test, sr.predict(X_test)))

In [ ]:
#neural_network.MLPRegressor
#X_train_neural, X_test_neural, y_train_neural, y_test_neural - StandardScaler вместо MinMaxScaler
#При MinMaxScaler
#best params {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': [10, 10], 'max_iter': 100, 'solver': 'lbfgs'}
#best score 0.5006479540511721
#30mins+
#При StandardScaler
#best params {'activation': 'logistic', 'alpha': 0.01, 'hidden_layer_sizes': [50, 50], 'max_iter': 200, 'solver': 'adam'}
#best score 0.5059130417591382
#3.1min finished
#При StandardScaler 2try
#best params {'activation': 'logistic', 'alpha': 0.1, 'hidden_layer_sizes': [50, 50], 'max_iter': 500, 'solver': 'adam'}
#best score 0.5069430997194276
# 28.2min
parameters = {\
              #'max_iter':[100,500,1000,2000],\ - исп.со standardScaler 
              #'max_iter':[100,200,50],\ - исп.1try standardScaler
              'max_iter':[1000,200,500],\ #-2try standardScaler
              #'solver':['lbfgs','adam','sgd'],\
              'solver':['adam'],\ #-2try standardScaler
              #'alpha':[0.0001,0.0005,0.001,0.01],\ - исп.со standardScaler 
              #'alpha':[0.0001,0.001,0.01],\ - исп.1try standardScaler
              'alpha':[10,1,0.01, 0.1],\
              #'hidden_layer_sizes':[[10],[50],[100],[200],[10,10],[50,50],[100,100],[200,200]],\ - исп.со standardScaler 
              #'hidden_layer_sizes':[[10],[50],[100],[10,10],[50,50]],\ - исп.1try standardScaler
              'hidden_layer_sizes':[[100,100],[50,50],[200,2000]],\
              'activation':['logistic', 'tanh']} #-2try standardScaler
mlpr = MLPRegressor(random_state = 33)
mlpr_grid = GridSearchCV(mlpr, parameters,
cv=5, n_jobs=-1,
verbose=True)
mlpr_grid.fit(X_train_neural, y_train_neural)
print('best params',mlpr_grid.best_params_)
print('best score',mlpr_grid.best_score_)


# mlpr =MLPRegressor(random_state = 33, max_iter = 500, solver = 'lbfgs')
# mlpr.fit(X_train, y_train)
# print("Правильность на обучающем наборе: {:.3f}".format(mlpr.score(X_train, y_train)))
# print("Правильность на тестовом наборе: {:.3f}".format(mlpr.score(X_test, y_test)))
# print('mse:', mean_squared_error(y_test, mlpr.predict(X_test)))

In [ ]:
#SVR,LinearSVR
#X_train_neural, X_test_neural, y_train_neural, y_test_neural - StandardScaler вместо MinMaxScaler
parameters = {"kernel":["linear", "poly", "rbf", "sigmoid", "precomputed"],'gamma':[0.001, 0.01, 0.1, 1, 10, 100, 1000],\
             'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
svr = SVR()
svr_grid = GridSearchCV(svr, parameters,
cv=5,
n_jobs=-1,
verbose=True)
svr_grid.fit(X_train_neural, y_train_neural)
print('best params',svr_grid.best_params_)
print('best score',svr_grid.best_score_)

#svr = SVR()
#svr.fit(X_train_neural, y_train_neural)
# print("Правильность на обучающем наборе: {:.3f}".format(svr.score(X_train_neural, y_train_neural)))
# print("Правильность на тестовом наборе: {:.3f}".format(svr.score(X_test_neural, y_test_neural)))
# print('mse:', mean_squared_error(y_test_neural, svr.predict(X_test_neural)))